In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
project_path = "/content/drive/MyDrive/Deep Learning/U-Net"
os.chdir(project_path)
print("Current directory:", os.getcwd())

In [ ]:
import torch
from u_net import U_Net
from data import get_data, view
from loops import training_loop, performance_report

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_data = get_data('train', [2, 7, 10, 12, 21, 24, 27, 28, 30, 43])
val_data = get_data('val', [1, 11, 22, 32])
test_data = get_data('test', [4, 16, 29, 36])

In [4]:
model = U_Net(
    in_channels = 3, 
    out_channels = 2, 
    start_channels = 64, 
    depth = 4, 
    bilinear = False
)

In [ ]:
model = training_loop(
    model = model, 
    training_data = train_data, 
    validation_data = val_data, 
    batch = 2, 
    learning_rate = 1e-3, 
    num_epochs = 10, 
    device = device
)

torch.torch.cuda.empty_cache()